In [1]:
import os
import plotly
import geojson
import imageio
import numpy as np
import pandas as pd
import networkx as nx
import plotly.plotly as py
import moviepy.editor as mp
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from itertools import groupby
from plotly.offline import iplot, init_notebook_mode

In [2]:
init_notebook_mode(connected=True)

mapbox_access_token = 'pk.eyJ1Ijoic2lsdmVyc2VlZCIsImEiOiJjam5pNmc3NzAwN3VkM3BsazM3cHRzcmRjIn0.3H4Qchxudkfs8D1PWxLUvA'
plotly.tools.set_credentials_file(username = 'fansantos', api_key = 'I9HKCsAXlgB1hEMlqcYG')
#Danillo: username = 'dbsdmat', api_key='lhTPNNiRf2Ala96MNKp3'
#Everlon: username = 'GoldSeed', api_key = '1cuptk54PTYZG8SnX1W2'
#Fernando: username = 'fansantos', api_key = 'I9HKCsAXlgB1hEMlqcYG'

plotly.io.orca.config.mapbox_access_token = mapbox_access_token
plotly.io.orca.config.save()

tbA ='https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Tabalas-oficiais/Tabela_Oficial_2013.csv'

In [3]:
def Casos(arq,semana):
    """Lê o arquivo do ano com informação dos casos epidêmicos e semana de ocorrência. Retorna um Dataframe com as informações de ocorrêcias por bairro."""
    tabela_oficial=pd.read_csv('lat e long recife.csv' ,sep='\t',parse_dates = True, na_values = 0, encoding='latin-1',low_memory=False).rename(index=str, columns={"co_bairro_residencia": "codigo", "no_bairro_residencia": "bairro","lat":"latitude","long":"longitude"})
    T=pd.read_csv(arq, parse_dates = True, na_values = 0, encoding='latin-1',low_memory=False)
    L=T[T['semana']==semana]
    bairro=list(L['bairro'])
    #data=dict([(i, bairro.count(i)) for i in tabela_oficial['bairro']])
    #Funciona da mesma forma que:
    data={}
    for i in tabela_oficial['bairro'].map(lambda x: x.strip()):
        data[i]=[bairro.count(i)]
    return pd.DataFrame(data)  

def correlation_matrix(arq,t0,t1,e=2,absolute=True):
    """Recebe informações de ano(s), semana inicial e final, valor de Euler (por padrão igual a 2) e método (por padrão, absoluto). Se absolute=False, considera-se apenas valores de correlação maior que 0, se 'original', retorna-se as correlações positivas, nulas e negativas."""
    C=Casos(arq,t0)
    for i in range(t0,t1):
        C.loc[i]=Casos(arq,i+1).loc[0]
    if absolute=='original':
        matrix=np.array(C.corr().fillna(0))
        filter_matrix=[[0 if matrix[i,j]<1-e else matrix[i,j] for i in range(0,94)] for j in range(0,94)] 
    else:
        if absolute==True:
            matrix=np.absolute(np.array(C.corr().fillna(0)))
            filter_matrix=[[0 if matrix[i,j]<1-e else matrix[i,j] for i in range(0,94)] for j in range(0,94)]
        else:
            matrix=np.array(C.corr().fillna(0))
            M=np.array([[0 if matrix[i,j]<0 else matrix[i,j] for i in range(0,94)] for j in range(0,94)])
            filter_matrix=[[0 if M[i,j]<1-e else M[i,j] for i in range(0,94)] for j in range(0,94)]
  
    return pd.DataFrame(filter_matrix)

def graph_from_matrix(arq,t0,t1,e,absolute=True,p=False):
    """Retorna o gráfico a partir da função de correlação correlation_matrix. Por padrão, o gráfico não é desenhado (p=False)."""
    tabela=pd.read_csv('lat e long recife.csv' ,sep='\t',parse_dates = True, na_values = 0, encoding='latin-1',low_memory=False).rename(index=str, columns={"co_bairro_residencia": "codigo", "no_bairro_residencia": "bairro","lat":"latitude","long":"longitude"})
    #tabela=pd.read_csv('Tabela_Oficial_2014.csv',na_values=0)
    tabela.fillna(0)
    Graph=nx.from_numpy_matrix(np.array(correlation_matrix(arq,t0,t1,e,absolute)))
    mapping={i:i+1 for i in range(0,94)}
    H=nx.relabel_nodes(Graph, mapping)
    for i in range(0,94):
        H.add_node(i+1, pos=(tabela['longitude'][i], tabela['latitude'][i]))
        H.node[i+1]['codigo']=tabela['codigo'][i]
        H.node[i+1]['bairro']=tabela['bairro'][i]
        H.node[i+1]['latitude']=tabela['latitude'][i]
        H.node[i+1]['longitude']=tabela['longitude'][i]
    H.nodes(data=True)  
    #Graph=nx.from_numpy_matrix(np.array(correlation_matrix(arq,t0,t1,e)))
    #nx.draw(Graph, nx.get_node_attributes(F, 'pos'), with_labels=True, node_size=200)
    if p==True:
        nx.draw(H, nx.get_node_attributes(H, 'pos'), with_labels=True, node_size=200)
        plt.show()
    return H
#graph_from_matrix('2015.csv',1,52,0.2,'original',True)

In [4]:
def plotly_graph(arq ,t0 ,t1 ,e ,absolute = True ,user = 'fansantos' ,api = 'I9HKCsAXlgB1hEMlqcYG'):
    """Gera o gráfico do pacote plotly a partir do gráfico gerado por graph_from_matrix. Lembre-se que você precisará criar uma conta no Plotly e exibir seu login e api_key para esta função."""
   
    G=graph_from_matrix(arq,t0,t1,e,absolute)
    
    mapbox_access_token = 'pk.eyJ1Ijoic2lsdmVyc2VlZCIsImEiOiJjam5pNmc3NzAwN3VkM3BsazM3cHRzcmRjIn0.3H4Qchxudkfs8D1PWxLUvA'
    geojson = 'https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Tabalas-oficiais/Fronteira%20dos%20Bairros.json'
    plotly.tools.set_credentials_file(username = user, api_key = api)
    #Danillo: username = 'dbsdmat', api_key='lhTPNNiRf2Ala96MNKp3'
    #Everlon: username = 'GoldSeed', api_key = '1cuptk54PTYZG8SnX1W2'
    #Fernando: username = 'fansantos', api_key = 'I9HKCsAXlgB1hEMlqcYG'
    
    df=pd.read_csv('https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Dengue/Dados%20de%20Recife/Tabela_de_Bairros.csv' ,sep=',',parse_dates = True, na_values = 0, encoding='latin-1',low_memory=False).rename(index=str, columns={"co_bairro_residencia": "codigo", "no_bairro_residencia": "bairro","lat":"latitude","long":"longitude"})
    df = df.fillna(0)
    Title = 'Grafo de '+str(arq[:-4])+'<br> Semanas de '+str(t0)+' a '+str(t1)+'<br> Parâmetro e = '+str(e)
    
    edge_trace = go.Scattermapbox(
        lon=[],
        lat=[],
        mode='lines',
        line=dict(width=1.5,color='crimson'), 
        opacity = 0.3,
        hoverinfo='none')

    for edge in G.edges():
        x0, y0 = G.node[edge[0]]['pos']
        x1, y1 = G.node[edge[1]]['pos']
        edge_trace['lon'] += tuple([x0, x1, None])
        edge_trace['lat'] += tuple([y0, y1, None])

        
    node_trace = go.Scattermapbox(
        lon=[],
        lat=[],
        text=[str(i) for i in df['codigo']],
        mode='markers+text',
        #hoverinfo='text',
        marker=dict(
            opacity = 0.9,
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='Electric',
            reversescale=True,
            color=[G.degree(i) for i in range(1,len(G.degree)+1)],
            cmin = 0,
            cmax = 94,
            size= 20, #[(G.degree(i))**(0.9) for i in range(1,len(G.degree)+1)], '''<-Colocar isso invés de 20 pra variar taman'''
            colorbar=dict(
                tickangle = -90,
                thickness = 25,
                title = 'Quantidade de Arestas',
                titleside = 'right',
                titlefont = dict(size = 14)
            )
        )
    )

    for node in G.nodes():
        x, y = G.node[node]['pos']
        node_trace['lon'] += tuple([x])
        node_trace['lat'] += tuple([y])
    
    #for i in range(0,len(G.degree)):
        #node_trace['marker']['color']+=tuple(list(G.degree)[i])
        #node_trace['marker']['size']+=tuple(list(G.degree)[i])
    
    Layout = go.Layout(
            title = Title,
            titlefont = dict(size = 16),
            showlegend=False,                     
            mapbox=dict(
                layers=[
                dict(
                    sourcetype = 'geojson',
                    source = geojson,
                    type = 'line',
                    line=dict(width=2),
                    opacity = 0.5,
                    color = 'gray'
                
                )
            ],
            accesstoken=mapbox_access_token,
            center=dict(
                 lat=-8.04,
                lon=-34.94
                ),
             zoom = 11
                )
            )
                     
    

    fig = go.Figure(data=[edge_trace, node_trace], layout = Layout)
    return fig

In [ ]:
iplot(plotly_graph('2015.csv' ,1 ,52 , 1))

In [ ]:
def plotly_graph(arq ,t0 ,t1 ,e ,absolute = True ,user = 'fansantos' ,api = 'I9HKCsAXlgB1hEMlqcYG'):
    """Gera o gráfico do pacote plotly a partir do gráfico gerado por graph_from_matrix. Lembre-se que você precisará criar uma conta no Plotly e exibir seu login e api_key para esta função."""
   
    G=graph_from_matrix(arq,t0,t1,e,absolute)
    
    mapbox_access_token = 'pk.eyJ1Ijoic2lsdmVyc2VlZCIsImEiOiJjam5pNmc3NzAwN3VkM3BsazM3cHRzcmRjIn0.3H4Qchxudkfs8D1PWxLUvA'
    geojson = 'https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Tabalas-oficiais/Fronteira%20dos%20Bairros.json'
    plotly.tools.set_credentials_file(username = user, api_key = api)
    #Danillo: username = 'dbsdmat', api_key='lhTPNNiRf2Ala96MNKp3'
    #Everlon: username = 'GoldSeed', api_key = '1cuptk54PTYZG8SnX1W2'
    #Fernando: username = 'fansantos', api_key = 'I9HKCsAXlgB1hEMlqcYG'
    
    df=pd.read_csv('https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Dengue/Dados%20de%20Recife/Tabela_de_Bairros.csv' ,sep=',',parse_dates = True, na_values = 0, encoding='latin-1',low_memory=False).rename(index=str, columns={"co_bairro_residencia": "codigo", "no_bairro_residencia": "bairro","lat":"latitude","long":"longitude"})
    df = df.fillna(0)
    Title = 'Grafo de '+str(arq[:-4])+'<br> Semanas de '+str(t0)+' a '+str(t1)+'<br> Parâmetro e = '+str(e)
    
    edge_trace = go.Scattermapbox(
        lon=[],
        lat=[],
        mode='lines',
        line=dict(width=1.5,color='crimson'), 
        opacity = 0.3,
        hoverinfo='none')

    for edge in G.edges():
        x0, y0 = G.node[edge[0]]['pos']
        x1, y1 = G.node[edge[1]]['pos']
        edge_trace['lon'] += tuple([x0, x1, None])
        edge_trace['lat'] += tuple([y0, y1, None])

        
    node_trace = go.Scattermapbox(
        lon=[],
        lat=[],
        text=[str(i) for i in df['codigo']],
        mode='markers+text',
        #hoverinfo='text',
        marker=dict(
            opacity = 0.9,
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='Electric',
            reversescale=True,
            color=[G.degree(i) for i in range(1,len(G.degree)+1)],
            cmin = 0,
            cmax = 94,
            size= 20, #[(G.degree(i))**(0.9) for i in range(1,len(G.degree)+1)], '''<-Colocar isso invés de 20 pra variar taman'''
            colorbar=dict(
                tickangle = -90,
                thickness = 25,
                title = 'Quantidade de Arestas',
                titleside = 'right',
                titlefont = dict(size = 14)
            )
        )
    )

    for node in G.nodes():
        x, y = G.node[node]['pos']
        node_trace['lon'] += tuple([x])
        node_trace['lat'] += tuple([y])
    
    #for i in range(0,len(G.degree)):
        #node_trace['marker']['color']+=tuple(list(G.degree)[i])
        #node_trace['marker']['size']+=tuple(list(G.degree)[i])
    
    Layout = go.Layout(
            title = Title,
            titlefont = dict(size = 16),
            showlegend=False,                     
            mapbox=dict(
                layers=[
                dict(
                    sourcetype = 'geojson',
                    source = geojson,
                    type = 'line',
                    line=dict(width=2),
                    opacity = 0.5,
                    color = 'gray'
                
                )
            ],
            accesstoken=mapbox_access_token,
            center=dict(
                 lat=-8.04,
                lon=-34.94
                ),
             zoom = 11
                )
            )
                     
    

    fig = go.Figure(data=[edge_trace, node_trace], layout = Layout)
    return fig

In [6]:
os.getcwd()

'C:\\Users\\Everl\\Documents\\Python Scripts\\Definitivus'

In [7]:
es = [i/100 for i in range(0,201)]

In [8]:
def Giffidaisso(arq ,t0 ,t1 , es, Pasta ,absolute = True):
    
    if not os.path.exists(Pasta):
        os.mkdir(Pasta)  
        
    frames=[plotly_graph(arq, t0, t1, es[i], absolute)
                        for i in range(len(es)+1)]
    [py.image.save_as(frames[i], Pasta+'\frame'+str(i)+'.png', width=900, height=900, scale=1.1)
                        for i in range(len(es)+1)]
    
    images=[imageio.imread(Pasta+'\frame'+str(i)+'.png')
                        for i in range(len(es)+1)]
    
    os.mkdir('Giff')  
    imageio.mimsave(Pasta+'\Giff\TomaTeuGiff.gif', images)

In [ ]:
Giffidaisso('2013.csv', 1, 52, es, '2013_Teste')

In [ ]:
clip = mp.VideoFileClip("2013_Teste\Giff\TomaTeuGiff.gif")

In [ ]:
clip.write_videofile("2013_Teste\Giff\TomaTeuGiff.mp4")

In [32]:
help(imageio.mimsave)

Help on function mimwrite in module imageio.core.functions:

mimwrite(uri, ims, format=None, **kwargs)
    mimwrite(uri, ims, format=None, **kwargs)
    
    Write multiple images to the specified file.
    
    Parameters
    ----------
    uri : {str, pathlib.Path, file}
        The resource to write the images to, e.g. a filename, pathlib.Path
        or file object, see the docs for more info.
    ims : sequence of numpy arrays
        The image data. Each array must be NxM, NxMx3 or NxMx4.
    format : str
        The format to use to read the file. By default imageio selects
        the appropriate for you based on the filename and its contents.
    kwargs : ...
        Further keyword arguments are passed to the writer. See :func:`.help`
        to see what arguments are available for a particular format.

